# Importing Libraries

In [ ]:
import os
import cv2 as cv
from skimage.io import imread,imshow,imsave
from skimage import img_as_ubyte
from skimage.color import rgba2rgb,rgb2gray
from skimage.filters import threshold_otsu,gaussian,laplace,threshold_multiotsu
from skimage.morphology import binary_closing,closing,dilation,erosion,opening,reconstruction,area_closing,area_opening
from skimage.measure import label, regionprops, regionprops_table
from skimage.feature import canny
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rembg import remove
from PIL import Image

from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import MinMaxScaler,StandardScaler

# Defining Folders

In [2]:
input_folder = "RGB/"
output_folder = "BW/"
extract_folder = "extract_BW/"
leaf_bw = "leaf_bw/"

# Image resize and pre process

In [3]:
def img_resize(image):
    shp = image.shape
    height = shp[0]
    width = shp[1]
    aspect_ratio = float(height) / float(width)
    new_height = int(2500 * aspect_ratio)
    resized_image = cv.resize(image,(2500 ,new_height))
    return resized_image


def image_process(image):
    img = imread(image)
    img = img_resize(img)
    rem_img = remove(img)
    rgb_img = rgba2rgb(rem_img)
    gray_img = rgb2gray(rgb_img)
    bin_img = gray_img < threshold_otsu(gray_img)
    smooth_img = gaussian(bin_img)

    seed_px = np.copy(smooth_img)
    seed_px[1:-1 , 1:-1]=smooth_img.max()
    mask = smooth_img
    filled_img = reconstruction(seed_px , mask , method ='erosion')
    filled_img = img_as_ubyte(filled_img)

    return filled_img

# Extract shape features and putting into dataframe

In [ ]:
leaf_classes = os.listdir(extract_folder)
df = pd.DataFrame()
sp = 1

for clas in leaf_classes:
    leafpath= extract_folder + clas +"/"
    leaves = os.listdir(leafpath)
    print(clas)
    lf = 1
    for leaf in leaves:
        leaf_dir = leafpath + str(leaf)
        leaf_img = imread(leaf_dir)
        leaf_img = leaf_img > 0
        label_img = label(leaf_img)
        
        table = pd.DataFrame( regionprops_table(label_img , leaf_img , properties=
                             ['convex_area', 'area', 'eccentricity',
                           'extent', 'perimeter' ,
                              'solidity',
                           'orientation','major_axis_length','minor_axis_length',
                           'equivalent_diameter',
                              'centroid'] ))
        table['perimeter_area_ratio'] = table['perimeter']/table['area']
        
        species = []
        species_no = []
        leaf_no = []
        
        for prop in regionprops(label_img):
            species += [str(clas)]
            species_no += [str(sp)]
            leaf_no += [str(lf)]
        table['species'] = species
        table['species_no'] = species_no
        table['leaf_no'] = leaf_no
        lf += 1

        df = pd.concat([df, table], axis=0)
    sp += 1


df.to_csv(r'feature.csv', sep= ',' , index=True , encoding='utf-8')
df1 = pd.read_csv("feature.csv")

# test train splittting

In [ ]:
X = df1.drop(columns=['species', 'leaf_no','species_no'])

#features
X = X[['area','extent','perimeter','minor_axis_length',
       'major_axis_length','perimeter_area_ratio',
       'minor_axis_length', 'solidity', 'eccentricity','equivalent_diameter']]

#scaler = MinMaxScaler()
scaler = StandardScaler()
X = scaler.fit_transform(X)
#print(X)
#target
y = df1['species_no']
#columns = X.columns

#train-test-split
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.3, random_state=1, stratify=y)

# Classifiers

In [ ]:
#GBM
clf = GradientBoostingClassifier(n_estimators=50, max_depth=3, random_state=123)
clf.fit(X_train, y_train)

#print confusion matrix of test set
print(classification_report(clf.predict(X_test), y_test))

#print accuracy score of the test set
print(f"Test Accuracy: {np.mean(clf.predict(X_test) == y_test)*100:.2f}%")

#Decision Tree
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
print(classification_report(clf.predict(X_test), y_test))
print(f"Test Accuracy: {np.mean(clf.predict(X_test) == y_test)*100:.2f}%")

#KNN
for i in range(1,50):
    clf = KNeighborsClassifier(n_neighbors=i)
    clf.fit(X_train, y_train)
    print(classification_report(clf.predict(X_test), y_test))
    print(f"Test Accuracy: {np.mean(clf.predict(X_test) == y_test)*100:.2f}%" , i)

    
#SVM   
clf = SVC(kernel='rbf' , random_state = 0)
clf.fit(X_train, y_train)
print(classification_report(clf.predict(X_test), y_test))
print(f"Test Accuracy: {np.mean(clf.predict(X_test) == y_test)*100:.2f}%")


